## RNN(recurrent neural network, 순환신경망)
앞서 배운 신경망들은 모든 신호가 오직 출력층 방향으로만 향하고 있었다.  
이런 신경망들을 feedforward 신경망이라고 하는데, 한 시점의 이미지나 정보만을 이용한다는 단점이 있다.  


RNN은 시간에 따라 순차적으로 제공되는 정보를 다룰 수 있는 신경망으로, 다양한 분야에서 널리 이용되고 있다.  
RNN을 구성하는 뉴런들은 활성화함수를 통해 나온 출력이 다시 자기자신에게 입력으로 제공되는 구조이다.  
이런 뉴런은 시간의 흐름에 따라 연속적으로 발생하는 신호를 보고, 다음 신호를 예측하는 일에 좋은 성능을 보인다.  
RNN의 대표적 응용사례는 네이버의 파파고 같은 기계번역 프로그램이다.  
입력과 출력 모두 sequence로, 글자나 단어들이 순서를 가지고 연속적으로 나타난다.  
1:m, n:1, n:m 의 다양한 기법이 있다.


+ 첫번째 cell : x1 입력 => h1 출력
+ 두번째 cell : x2 입력 + h1 => h2
+ 세번째 cell : x3 입력 + h2 => h3 ....


모든 layer(=순환cell)는 이전 상태 정보를 보존하는 기능과, 다음 층으로 정보를 전달하는 구조를 가지며,  
신호가 전달될 때는 학습을 통해 변경되는 weight가 곱해져서 전달된다.


하나의 셀은 (자신의 이전 셀에서 얻은 입력 + 새롭게 제공된 입력)을 하나의 활성화함수에 제공한다.  
활성화함수는 일반적으로 tanh를 사용하며, 그 결과는 노드의 출력 h가 되어 순환edge를 통해 자신에게 feedback된다.
+ tanh를 사용하는 이유?
    + ReLU를 사용하면 feedback 구조 때문에 신호가 지나치게 커지는 overflow가 발생할 수 있다.
    + tanh, sigmoid의 경우 값이 특정 범위를 넘지 못하게 되어있어 위 문제를 피할 수 있다.
    + tanh가 sigmoid보다 기울기소실 문제를 더 잘 해결한다.

### Tensorflow로 단순 RNN 모델 만들기
+ 입력 : [0.0, 0.1, 0.2] 처럼 3개의 원소를 가지는 시퀀스 데이터
+ 목표값 : 0.3 처럼 하나의 실수 레이블 값

In [7]:
import numpy as np

size, seq_len = 100, 3

X = np.empty(shape=(size, seq_len, 1))   #(100, 3, 1)
Y = np.empty(shape=(size,))              #(100,)

for i in range(size):
    c = np.linspace(i/10, (i+seq_len-1)/10.0, seq_len)   #(3, )
    X[i] = c[:, np.newaxis]   #(3, 1)
    Y[i] = (i+seq_len)/10
    
for i in range(5):
    print(X[i], Y[i])

[[0. ]
 [0.1]
 [0.2]] 0.3
[[0.1]
 [0.2]
 [0.3]] 0.4
[[0.2]
 [0.3]
 [0.4]] 0.5
[[0.3]
 [0.4]
 [0.5]] 0.6
[[0.4]
 [0.5]
 [0.6]] 0.7


In [8]:
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([keras.layers.SimpleRNN(units=20, input_shape=[3,1],  #layer에 있는 뉴런 유닛의 수, 입력형태는 [seq_len,1]로 각각이 모두 출력을 발생시킴
                                                 return_sequences=False),      #출력으로 시간시리즈에 걸친 시퀀스 전체를 출력할지
                          keras.layers.Dense(1)])   #20개의 유닛의 각 출력들을 하나의 레이블 값으로 만들기 위해 출력노드 개수를 1로 지정

model.compile(optimizer='adam', loss='mse')

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 20)                440       
_________________________________________________________________
dense (Dense)                (None, 1)                 21        
Total params: 461
Trainable params: 461
Non-trainable params: 0
_________________________________________________________________


(유닛 20개 x 순환에지 20개) + 입력을 각 유닛에 보내는 연결 20개 + 각 유닛의 출력을 위한 연결 20개  
=> 이 신경망을 위한 파라미터는 총 440개

이 계층의 출력 20개 + 편향 1개  
=> 출력을 위한 연결망을 위한 파라미터는 총 21개  


=> total params : 461